In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import image_lib as imlib

## Gradient Method

In [4]:
lena = cv2.imread("../Lab_1/assets/Lena.jpg", 0)

cv2.imshow("Lena", lena)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Sobel Kernel

In [3]:
gx = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
gy = np.array([[-1, -1, -1], [0, 0, 0], [1, 1,1]])

In [4]:
gx_convolved = cv2.filter2D(lena, -1, gx)
gy_convolved = cv2.filter2D(lena, -1, gy)

cv2.imshow("Gradient X", gx_convolved)
cv2.imshow("Gradient Y", gy_convolved)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
gxc_float = gx_convolved.astype(np.float64)
gyc_float = gy_convolved.astype(np.float64)
magnitude = np.sqrt(gxc_float**2 + gyc_float**2)

magnitude_norm = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

cv2.imshow("Gradient Magnitude", magnitude_norm)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Standard Thresholding

In [6]:
# Manual Implementation
def manual_binary_threshold(image, threshold_value):
    binary_image = np.zeros_like(image)
    pixels_above_threshold = image > threshold_value
    binary_image[pixels_above_threshold] = 255
    
    return binary_image

In [7]:
threshold_value = 50 
# Using OpenCV's threshold function
_, binary_edges = cv2.threshold(magnitude_norm, threshold_value, 255, cv2.THRESH_BINARY)
# Using a manual thresholding function
binary_edges_2 = manual_binary_threshold(magnitude_norm, threshold_value)

cv2.imshow("Binary Edges", binary_edges)
cv2.imshow("Binary Edges 2", binary_edges_2)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Hysteresis Thresholding 

In [8]:
def manual_hysteresis_thresholding(image, low_thresh, high_thresh):
    """
    Manually performs hysteresis thresholding using NumPy.

    Args:
        image (numpy.ndarray): The input gradient magnitude image.
        low_thresh (int): The low threshold for classifying weak pixels.
        high_thresh (int): The high threshold for classifying strong pixels.

    Returns:
        numpy.ndarray: The final binary edge map.
    """
    # --- 1. Double Thresholding ---
    # Define values for strong and weak pixels
    strong_pixel_val = 255
    weak_pixel_val = 75

    # Create an output image to store the results
    output = np.zeros_like(image, dtype=np.uint8)

    # Find coordinates of strong and weak pixels using boolean indexing
    strong_pixels = image >= high_thresh
    weak_pixels = (image <= high_thresh) & (image >= low_thresh)

    # Assign strong and weak values to the output image
    output[strong_pixels] = strong_pixel_val
    output[weak_pixels] = weak_pixel_val

    # --- 2. Edge Tracking by Hysteresis ---
    rows, cols = image.shape
    # Iterate through the image to connect weak pixels to strong ones
    # We iterate multiple times to allow chains of weak pixels to connect
    for _ in range(5): # Iterate a few times to propagate connections
        for r in range(1, rows - 1):
            for c in range(1, cols - 1):
                # If the current pixel is a weak pixel
                if output[r, c] == weak_pixel_val:
                    # Check its 8-pixel neighborhood
                    neighborhood = output[r-1:r+2, c-1:c+2]
                    # If any neighbor is a strong pixel, promote the current weak pixel
                    if np.max(neighborhood) == strong_pixel_val:
                        output[r, c] = strong_pixel_val

    # --- 3. Cleanup ---
    # Set any remaining weak pixels that were not connected to 0
    output[output == weak_pixel_val] = 0
    
    return output

In [9]:
low_threshold = 30
high_threshold = 100
hysteresis_edges = manual_hysteresis_thresholding(magnitude_norm, low_threshold, high_threshold)

cv2.imshow("Hysteresis Edges", hysteresis_edges)

cv2.waitKey(0)
cv2.destroyAllWindows()


## Laplacian Method

In [10]:
lena = cv2.imread("../Lab_1/assets/Lena.jpg", 0)

cv2.imshow("Lena", lena)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Manual Implementation

In [11]:
def manual_laplacian_edge_detection(image, kernel, threshold_value=10):
    """
    Manually applies the Laplacian operator using a specified kernel.

    Args:
        image (numpy.ndarray): The input grayscale image.
        kernel (numpy.ndarray): The Laplacian kernel to apply.
        threshold_value (int): The threshold to create the final binary map.

    Returns:
        tuple: A tuple containing the raw Laplacian response and the thresholded edges.
    """
    # --- 1. Noise Reduction (Crucial for Laplacian) ---
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # --- 2. Manual Convolution ---
    # Apply the user-defined kernel using cv2.filter2D
    # cv2.CV_64F is used to handle negative values in the output
    laplacian_response = cv2.filter2D(src=blurred, ddepth=cv2.CV_64F, kernel=kernel)

    # --- 3. Process Output for Display and Thresholding ---
    # Take the absolute value to see the magnitude of the response
    abs_laplacian = np.absolute(laplacian_response)
    
    # Convert to an 8-bit image for display
    laplacian_display = cv2.convertScaleAbs(abs_laplacian)
    
    # Create the final binary edge map by thresholding
    _, binary_edges = cv2.threshold(laplacian_display, threshold_value, 255, cv2.THRESH_BINARY)
    
    return laplacian_display, binary_edges

- Robust Laplacian

In [12]:
laplacian_kernel = np.array([
    [1,  1,  1],
    [1, -8,  1],
    [1,  1,  1]
], dtype=np.float32)

laplacian_result, final_edges = manual_laplacian_edge_detection(lena, laplacian_kernel)

cv2.imshow("Laplacian", laplacian_result)
cv2.imshow("Final Edges", final_edges)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Library Implementation

In [13]:
# This is crucial to reduce noise before applying the Laplacian
blurred = cv2.GaussianBlur(lena, (5, 5), 0)

laplacian = cv2.Laplacian(blurred, cv2.CV_64F)

laplacian_display = cv2.convertScaleAbs(laplacian)

_, edges = cv2.threshold(laplacian_display, 10, 255, cv2.THRESH_BINARY)

cv2.imshow("Laplacian", laplacian)
cv2.imshow("Laplacian Display", laplacian_display)
cv2.imshow("Laplacian Edges", edges)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Canny Method

In [27]:
lena = cv2.imread("../Lab_1/assets/Lena.jpg", 0)

cv2.imshow("Lena", lena)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Using Library

In [28]:
blurred = cv2.GaussianBlur(lena, (5, 5), 0)

low_threshold = 50
high_threshold = 150

edges = cv2.Canny(image=blurred, threshold1=low_threshold, threshold2=high_threshold)

cv2.imshow("Canny Edges", edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Manual Implementation

In [29]:
def create_gaussian_kernel(sigma=1, size=None):
    if size is None:
        size = int(6 * sigma) + 1  # Kernel size should be odd
        if size % 2 == 0:
            size += 1
            
    kernel = np.zeros((size, size))
    center = size // 2
    for i in range(size):
        for j in range(size):
            x, y = i - center, j - center
            kernel[i, j] = (1 / (2 * np.pi * sigma**2)) * np.exp(-(x**2 + y**2) / (2 * sigma**2))
    return kernel / np.sum(kernel)

In [30]:
def convolve2d(image, kernel):
    k_h, k_w = kernel.shape
    pad_h, pad_w = k_h // 2, k_w // 2
    
    padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='edge')
    output = np.zeros_like(image)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            output[i, j] = np.sum(kernel * padded_image[i:i+k_h, j:j+k_w])
            
    return output

In [31]:
def non_maximum_suppression(magnitude, direction):
    """
    Perform non-maximum suppression on gradient magnitude using gradient direction.
    """
    M, N = magnitude.shape
    suppressed = np.zeros((M, N), dtype=np.float32)
    angle = np.rad2deg(direction) % 180

    for i in range(1, M-1):
        for j in range(1, N-1):
            try:
                q = 0  # Initialize to 0 instead of 255
                r = 0
                
                # Angle 0 degrees (horizontal edge)
                if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                    q = magnitude[i, j+1]
                    r = magnitude[i, j-1]
                # Angle 45 degrees (diagonal edge)
                elif 22.5 <= angle[i, j] < 67.5:
                    q = magnitude[i+1, j-1]
                    r = magnitude[i-1, j+1]
                # Angle 90 degrees (vertical edge)
                elif 67.5 <= angle[i, j] < 112.5:
                    q = magnitude[i+1, j]
                    r = magnitude[i-1, j]
                # Angle 135 degrees (diagonal edge)
                elif 112.5 <= angle[i, j] < 157.5:
                    q = magnitude[i-1, j-1]
                    r = magnitude[i+1, j+1]

                if magnitude[i, j] >= q and magnitude[i, j] >= r:
                    suppressed[i, j] = magnitude[i, j]
                else:
                    suppressed[i, j] = 0

            except IndexError:
                pass
                
    return suppressed

In [32]:
def manual_canny(image, sigma=1, low_threshold_ratio=0.05, high_threshold_ratio=0.15):
    """
    A manual implementation of the Canny edge detector.
    
    Args:
        image (numpy.ndarray): Input grayscale image.
        sigma (float): Standard deviation for the Gaussian kernel.
        low_threshold_ratio (float): Ratio for the low hysteresis threshold.
        high_threshold_ratio (float): Ratio for the high hysteresis threshold.
        
    Returns:
        numpy.ndarray: The final binary edge map.
    """
    # Ensure input is float for calculations
    if image is None:
        raise ValueError("Input image is None")
    
    image = image.astype(np.float64)
    
    # --- Step 1: Noise Reduction with Gaussian Blur ---
    gaussian_kernel = create_gaussian_kernel(sigma)
    blurred_image = convolve2d(image, gaussian_kernel)

    # --- Step 2: Gradient Calculation (Sobel) ---
    sobel_x_kernel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
    sobel_y_kernel = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float32)
    
    gradient_x = convolve2d(blurred_image, sobel_x_kernel)
    gradient_y = convolve2d(blurred_image, sobel_y_kernel)

    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    gradient_direction = np.arctan2(gradient_y, gradient_x)
    
    # Handle NaN and infinite values
    gradient_magnitude = np.nan_to_num(gradient_magnitude)
    gradient_direction = np.nan_to_num(gradient_direction)

    # --- Step 3: Non-Maximum Suppression ---
    suppressed_image = non_maximum_suppression(gradient_magnitude, gradient_direction)

    # --- Step 4 & 5: Double Thresholding and Hysteresis ---
    max_val = np.max(suppressed_image)
    if max_val > 0:
        high_thresh = high_threshold_ratio * max_val
        low_thresh = low_threshold_ratio * max_val
    else:
        high_thresh = 1
        low_thresh = 0.5
    
    # Apply hysteresis thresholding
    final_edges = manual_hysteresis_thresholding(suppressed_image, low_thresh, high_thresh)

    return final_edges

In [ ]:
manual_canny_img = manual_canny(lena)

In [36]:
cv2.imshow("Library Canny Edges", edges)
cv2.imshow("Manual Canny Edges", manual_canny_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Lab Practise

In [ ]:
lena2 = cv2.imread("../Lab_1/assets/Lena.jpg", 1)

def laplacian_of_gaussian(image, sigma=1.0):
    gray_image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
    kernel = imlib.Gaussian_Sharpenning_kernel(7,sigma)
    log_img = imlib.convolve(gray_image,kernel)
    return log_img

def zero_crossing(log_img):
    zc = np.zeros(log_img.shape, dtype=np.uint8)
    h,w = log_img.shape
    for i in range(1, h-2):
        for j in range(1, w-2):
            patch = log_img[i-1:i+2, j-1:j+2]
            local_std = np.std(patch)
            if np.max(patch) > 0 and np.min(patch) < 0:
                zc[i, j] = log_img[i,j]
                
    return zc

def local_variance(image, ksize=5):
    mean = cv2.blur(image, (ksize, ksize))
    mean_sq = cv2.blur(image**2, (ksize, ksize))
    variance = mean_sq - mean**2
    return variance

def robust_laplacian_edge_detector(image, sigma=1.0, var_thresh=100):
    log_img = laplacian_of_gaussian(image, sigma)
    zc_img = zero_crossing(log_img)
    imlib.show_image(zc_img, 'Zero Crossing Image')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    variance = local_variance(gray, ksize=5)
    edge_img = np.zeros_like(zc_img)
    edge_img[(zc_img == 255) & (variance > var_thresh)] = 255
    return edge_img

edge_points = robust_laplacian_edge_detector(lena2, sigma=1, var_thresh=160)
imlib.show_image(edge_points, 'Robust Laplacian Edge Detector')

In [14]:
# Robust Laplacian-based Edge Detector Implementation following slides exactly

def laplacian_of_gaussian_slides(image, sigma=1.0):
    """Apply LoG operator as shown in slides"""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = imlib.Gaussian_Sharpenning_kernel(7, sigma)
    log_img = imlib.convolve(gray_image, kernel)
    return log_img

def zero_crossing_detection_slides(log_img):
    """Zero crossing detection following slide example"""
    h, w = log_img.shape
    zc = np.zeros((h, w), dtype=np.uint8)
    
    for i in range(1, h-1):
        for j in range(1, w-1):
            # Extract 3x3 local region
            local_region = log_img[i-1:i+2, j-1:j+2]
            
            # Check for zero crossing (min < 0 and max > 0)
            min_val = np.min(local_region)
            max_val = np.max(local_region)
            
            if min_val < 0 and max_val > 0:
                zc[i, j] = log_img[i, j]
    
    return zc

def region_of_interest_std(log_img, threshold=2.0):
    """Apply standard deviation check in region of interest"""
    h, w = log_img.shape
    roi_mask = np.zeros((h, w), dtype=np.uint8)
    pad = 1
    
    for i in range(pad, h-pad):
        for j in range(pad, w-pad):
            # Extract local region as shown in slides
            local_region = log_img[i-pad:i+pad+1, j-pad:j+pad+1]
            local_stddev = np.std(local_region)
            
            if local_stddev > threshold:
                roi_mask[i, j] = 255
    
    return roi_mask

def estimate_local_variance_slides(image, ksize=5):
    """Estimate local variance (σ²)"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    mean = cv2.blur(gray.astype(np.float64), (ksize, ksize))
    mean_sq = cv2.blur((gray.astype(np.float64))**2, (ksize, ksize))
    variance = mean_sq - mean**2
    return variance

def robust_laplacian_slides(image, sigma=1.0, var_thresh=60):
    """Complete Robust Laplacian following algorithm flowchart"""
    
    # Step 1: Apply LoG operator
    log_response = laplacian_of_gaussian_slides(image, sigma)
    
    # Step 2: Zero crossing detection
    zero_crossings = zero_crossing_detection_slides(log_response)

    # Show zero crossing image
    imlib.show_image(zero_crossings, 'Zero Crossings')
    
    # Step 3: Estimate local variance
    local_variance = estimate_local_variance_slides(image, ksize=5)
    
    # Step 4: Apply thresholding (σ² > threshold)
    final_edges = np.zeros_like(zero_crossings)
    final_edges[(zero_crossings == 255) & (local_variance > var_thresh)] = 255
    
    return final_edges

# Test with σ² > 60
lena2 = cv2.imread("../Lab_1/assets/Lena.jpg", 1)
edges_60 = robust_laplacian_slides(lena2, sigma=1.0, var_thresh=60)

imlib.show_image(edges_60, 'Robust Laplacian (σ² > 60)')